## Auto Regressive for Feature Extraction by Lags of time series 
 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline
import mne
mne.utils.use_log_level('error')

- Reading all files and selecting Channel 4 for each subject
- Take 5 sec segment for 1 min data and using autoregressive with 17 lags of time series values for each subject
- Build a dataframe of features according to subject



In [2]:
import glob

path = r'Preprocessed_Data'
all_files = glob.glob(path + "/*_Non-VR.fif")

li, s = [],[]
cols = []
sub = 1
col = ['Features']
for filename in all_files:
    raw =  mne.io.read_raw_fif(filename, preload=True, verbose='error');
    
    if 'CH 4' in raw.info['ch_names']:
        raw.pick_channels(["CH 4"])
        
        picks = mne.pick_types(raw.info, eeg=True, exclude=[])
        interval = 5
        start = 60
        stop = start + interval
        while(stop <= 120):
            start1, stop1 = raw.time_as_index([start, stop])
            data, times = raw[picks[:], start1:stop1]
            segment = pd.Series(data[0]).shift(17).dropna()
        
        #d = pd.Series(raw.get_data()[0]).shift(17).dropna()
            li.append(segment)
            for i in range(len(segment)):
                s.append(sub)

            start += interval
            stop += interval

            cols.append(s)
    sub += 1
    
s = pd.Series(s)      
frame = pd.concat(li, ignore_index=True, sort=False)

df = pd.DataFrame([frame,s])


In [3]:
df1 = df.T
df1.columns = ['Features','Subjects']
df1

,Features,Subjects
0,17.525848,1.0
1,6.133788,1.0
2,-22.865923,1.0
3,-40.482288,1.0
4,-31.190411,1.0
5,-35.478352,1.0
6,-37.260670,1.0
7,-15.887967,1.0
8,-8.499919,1.0
9,-11.071376,1.0


In [4]:
df1.shape

(216804, 2)

### Support Vectore Machine (SVM)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [6]:
X = df1.drop(['Subjects'], axis=1)
y = df1['Subjects']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [7]:
model = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto')

In [8]:
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [9]:
prediction = model.predict(X_test)

In [10]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

         1.0       0.09      0.23      0.13      2282
         2.0       0.00      0.00      0.00      2205
         3.0       0.04      0.01      0.02      2260
         4.0       0.04      0.04      0.04      2246
         5.0       0.04      0.05      0.05      2243
         6.0       0.05      0.04      0.05      2238
         7.0       0.05      0.30      0.09      2241
         8.0       0.04      0.02      0.03      2238
         9.0       0.00      0.00      0.00      2239
        10.0       0.30      0.29      0.30      2264
        11.0       0.05      0.10      0.06      2250
        12.0       0.05      0.00      0.01      2269
        13.0       0.04      0.03      0.04      2324
        14.0       0.72      0.46      0.56      2216
        15.0       0.00      0.00      0.00      2301
        16.0       0.06      0.01      0.02      2225
        17.0       0.04      0.03      0.03      2285
        18.0       0.00    

c:\users\sonal\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sonal\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sonal\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [40]:
set(y_test) - set(prediction)

{9.0, 19.0}